In [381]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.applications.densenet import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score
%matplotlib inline

In [382]:
data_TSLA = pd.read_csv('../datasets/TSLA.csv', index_col=[0], parse_dates=[0])
data_GOOG = pd.read_csv('../datasets/GOOG.csv', index_col=[0], parse_dates=[0])
data_META = pd.read_csv('../datasets/META.csv', index_col=[0], parse_dates=[0])
data_NFLX = pd.read_csv('../datasets/TSLA.csv', index_col=[0], parse_dates=[0])
data_APPL = pd.read_csv('../datasets/TSLA.csv', index_col=[0], parse_dates=[0])

In [383]:
data_TSLA.columns = data_TSLA.columns.str.lower()
data_GOOG.columns = data_GOOG.columns.str.lower()
data_META.columns = data_META.columns.str.lower()
data_NFLX.columns = data_NFLX.columns.str.lower()
data_APPL.columns = data_APPL.columns.str.lower()

In [384]:
def smape(A, F):
    return 100 / len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))


def metrics(y_true, y_pred):
    print('R2 Score:', r2_score(y_true=y_true, y_pred=y_pred))
    print('RMSE:', mean_squared_error(y_true=y_true, y_pred=y_pred, squared=False))
    print('MAPE:', mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred))
    return r2_score(y_true=y_true, y_pred=y_pred), mean_squared_error(y_true=y_true, y_pred=y_pred, squared=False), mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred)

In [385]:
def get_x_y_train_test(feature, test_size, window):
    np.random.seed(12345)
    close_prices = feature
    values = close_prices.values
    training_data_len = math.ceil(len(values) * (1 - test_size))
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(values.reshape(-1, 1))
    train_data = scaled_data[0: training_data_len, :]
    x_train = []
    y_train = []
    for i in range(window, len(train_data)):
        x_train.append(train_data[i - window:i])
        y_train.append(train_data[i])
    x_train, y_train = np.array(x_train), np.array(y_train)
    test_data = scaled_data[training_data_len - window:]
    x_test = []
    y_test = values[training_data_len:]
    for i in range(window, len(test_data)):
        x_test.append(test_data[i - window:i, 0])
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1)) #TODO способ подачи
    return scaler, x_test, x_train, y_test, y_train

In [386]:
def fit_actions(x_train, y_train, n_features):
    np.random.seed(12345)
    model = keras.Sequential()
    model.add(layers.LSTM(100, return_sequences=False, input_shape=(x_train.shape[1], n_features)))

    model.add(layers.Dense(1))
    model.summary()
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, batch_size=15, epochs=100)
    return model

In [387]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [388]:
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [389]:
tesla = data_TSLA['close'].values
google = data_GOOG['close'].shift(-1).values
meta = data_META['close'].values
netflix = data_NFLX['close'].values
apple = data_APPL['close'].values
out_tesla = data_TSLA['close'].shift(-1).values
# total_tmp = np.column_stack((tmp_1, tmp_2))
# features = pd.DataFrame(total_tmp, columns=['close_TSLA', 'close_APPL'], index=data_TSLA.index)

In [390]:
data_TSLA

,open,high,low,close,adj close,volume
Date,,,,,,
2018-01-16,22.502666,23.000000,22.320000,22.670668,22.670668,97114500
2018-01-17,22.698000,23.266666,22.650000,23.143999,23.143999,106552500
2018-01-18,23.044666,23.486668,22.916000,22.971333,22.971333,85287000
2018-01-19,23.000000,23.372667,22.840000,23.334667,23.334667,73324500
2018-01-22,23.293333,23.855333,23.280001,23.437332,23.437332,93156000
...,...,...,...,...,...,...
2023-01-09,118.959999,123.519997,117.110001,119.769997,119.769997,190284000
2023-01-10,121.070000,122.760002,114.919998,118.849998,118.849998,167642500
2023-01-11,122.089996,125.949997,120.510002,123.220001,123.220001,183810800


In [391]:
# in_seq1 = tmp_1
# in_seq2 = tmp_2
# in_seq3 = tmp_3
# in_seq4 = tmp_4
# in_seq5 = tmp_5
# out_seq = tmp_1

In [392]:
from sklearn.preprocessing import StandardScaler

tesla = tesla.reshape((len(tesla), 1))
google = google.reshape((len(google), 1))
meta = meta.reshape((len(meta), 1))
netflix = netflix.reshape((len(netflix), 1))
apple = apple.reshape((len(apple), 1))
out_tesla = out_tesla.reshape((len(out_tesla), 1))
# horizontally stack columns
# dataset = np.hstack((tesla, google, meta, netflix, apple, out_tesla))

tesla = data_TSLA
tesla_0 = data_TSLA['close'].values
tesla_1 = data_TSLA['open'].values
tesla_2 = data_TSLA['high'].values
tesla_3 = data_TSLA['low'].values
tesla_4 = data_TSLA['volume'].values
tesla_5 = data_TSLA['close'].shift(-1).values

tesla_0 = tesla_0.reshape((len(tesla_0), 1))
tesla_1 = tesla_1.reshape((len(tesla_1), 1))
tesla_2 = tesla_2.reshape((len(tesla_2), 1))
tesla_3 = tesla_3.reshape((len(tesla_3), 1))
tesla_4 = tesla_4.reshape((len(tesla_4), 1))
tesla_5 = tesla_5.reshape((len(tesla_5), 1))

scaler = StandardScaler()
tesla_1 = scaler.fit_transform(tesla_1)
tesla_2 = scaler.fit_transform(tesla_2)
tesla_3 = scaler.fit_transform(tesla_3)
tesla_4 = scaler.fit_transform(tesla_4)

dataset = np.hstack((tesla_1, tesla_2, tesla_3, tesla_4, tesla_5))
dataset

array([[-9.35969440e-01, -9.36635550e-01, -9.32693294e-01,
        -3.99054901e-01,  2.31439990e+01],
       [-9.34308664e-01, -9.34418381e-01, -9.29813180e-01,
        -2.94390917e-01,  2.29713330e+01],
       [-9.31361228e-01, -9.32589195e-01, -9.27491634e-01,
        -5.30217569e-01,  2.33346670e+01],
       ...,
       [-8.92545401e-02, -8.06676226e-02, -7.57285143e-02,
         5.62375498e-01,  1.23559998e+02],
       [-8.52584726e-02, -9.57998390e-02, -1.06362469e-01,
         4.02574954e-01,  1.22400002e+02],
       [-1.36356863e-01, -1.08271446e-01, -1.18581151e-01,
         5.24986791e-01,             nan]])

In [393]:
n_steps = 60
X, y = split_sequences(dataset, n_steps)
n_features = X.shape[2]

for i in range(len(X)):
    print(X[i], y[i])

[[-0.93596944 -0.93663555 -0.93269329 -0.3990549 ]
 [-0.93430866 -0.93441838 -0.92981318 -0.29439092]
 [-0.93136123 -0.9325892  -0.92749163 -0.53021757]
 [-0.93174099 -0.93353705 -0.92815493 -0.66287734]
 [-0.929247   -0.92952397 -0.92431477 -0.44295322]
 [-0.92323875 -0.928044   -0.92326747 -0.56687991]
 [-0.9263109  -0.93123119 -0.92761964 -0.5964726 ]
 [-0.9298875  -0.93430751 -0.93176235 -0.35480725]
 [-0.93372485 -0.93718985 -0.93216382 -0.72091495]
 [-0.93466008 -0.93339294 -0.93066849 -0.68636519]
 [-0.93166164 -0.93482301 -0.92840513 -0.69125572]
 [-0.93031828 -0.930433   -0.92664796 -0.44233775]
 [-0.9283401  -0.92850961 -0.92464642 -0.77775488]
 [-0.92979114 -0.93278321 -0.92937099 -0.8597461 ]
 [-0.9357257  -0.93692933 -0.9337406  -0.73344069]
 [-0.94295826 -0.94150225 -0.93926809 -0.6295918 ]
 [-0.93514755 -0.93608125 -0.93219291 -0.31673099]
 [-0.93269891 -0.93462901 -0.94444649  0.23975803]
 [-0.94595106 -0.94994969 -0.95599021  0.67543101]
 [-0.94810496 -0.95155713 -0.94

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [394]:
# tesla[tesla['close'] == 15.095333]

In [395]:
features_train = X[:1000]
target_train = y[:1000]
features_test = X[1000:]
target_test = y[1000:]
target_train

array([ 19.605333,  20.022667,  19.414   ,  19.179333,  19.556667,
        20.005333,  19.349333,  18.891333,  18.897333,  18.712667,
        19.032   ,  19.605333,  19.593332,  19.994667,  20.076668,
        18.963333,  19.606001,  20.184668,  20.131332,  20.456667,
        20.334667,  20.070667,  19.464666,  18.945333,  19.098667,
        18.969334,  18.454666,  18.966   ,  18.334   ,  18.604668,
        18.523333,  18.59    ,  18.917334,  19.448   ,  18.982   ,
        19.454666,  19.782667,  19.408667,  21.299999,  21.072666,
        21.177334,  22.139999,  22.851334,  22.985332,  23.848   ,
        23.878   ,  24.722   ,  23.503332,  24.148001,  23.167334,
        22.242001,  22.200666,  22.799999,  22.966667,  23.328667,
        22.863333,  22.337999,  20.724001,  20.610666,  20.593332,
        21.233999,  21.497999,  21.264   ,  21.114   ,  21.257999,
        20.673332,  21.512667,  21.59    ,  21.348667,  20.905333,
        20.213333,  19.828667,  20.582666,  20.443333,  19.812

In [396]:
model_per_day = fit_actions(features_train, target_train, n_features)
predictions_per_day = model_per_day.predict(features_test)
predictions_per_day = scaler.inverse_transform(predictions_per_day)
r2_per_day, rmse_per_day, mape_per_day= metrics(y_true=target_test, y_pred=predictions_per_day)

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 100)               42000     
                                                                 
 dense_18 (Dense)            (None, 1)                 101       
                                                                 
Total params: 42,101
Trainable params: 42,101
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
67/67 [==============================] - 2s 19ms/step - loss: 24862.3906
Epoch 2/100
67/67 [==============================] - 1s 19ms/step - loss: 25695.8984
Epoch 3/100
67/67 [==============================] - 1s 19ms/step - loss: 22503.9238
Epoch 4/100
67/67 [==============================] - 1s 20ms/step - loss: 20742.7988
Epoch 5/100
67/67 [==============================] - 1s 19ms/step - loss: 19437.5215
Epoch 6/100
67/67 [=

In [397]:
metrics(y_true=target_test, y_pred=model_per_day.predict(features_test))

7/7 [==============================] - 0s 9ms/step
R2 Score: 0.7314007560576907
RMSE: 31.387406922626834
MAPE: 0.12743975240695302


(0.7314007560576907, 31.387406922626834, 0.12743975240695302)